In [2]:
import pandas as pd
import plotly.express as px

In [9]:
df = pd.read_csv("aave_price_data.csv")
fig = px.line(df, x="Date", y="Price")

# template="plotly_dark"

fig.update_layout(
    xaxis = dict(
        tickmode = "array",
        tickvals = ['Mar 01, 2022', 'Feb 05, 2022', 'Jan 06, 2022', 'Dec 01, 2021',
                    'Nov 01, 2021', 'Oct 02, 2021', 'Sep 02, 2021', 'Aug 03, 2021',
                    'Jul 04, 2021', 'Jun 04, 2021'],
        ticktext = ['Mar 2022', 'Feb 2022', 'Jan 2022', 'Dec 2021',
                    'Nov 2021', 'Oct 2021', 'Sep 2021', 'Aug 2021',
                    'July 2021', 'June 2021']
    )
)

# # chart line color red
# fig.update_traces(line_color='#FF0000')

fig.show()

In [3]:
df = pd.read_csv("uniswap_price_data.csv")
fig = px.line(df, x="Date", y="Price")

# template="plotly_dark"

fig.update_layout(
    xaxis = dict(
        tickmode = "array",
        tickvals = ['Mar 01, 2022', 'Feb 05, 2022', 'Jan 06, 2022', 'Dec 01, 2021',
                    'Nov 01, 2021', 'Oct 02, 2021', 'Sep 02, 2021', 'Aug 03, 2021',
                    'Jul 04, 2021', 'Jun 04, 2021'],
        ticktext = ['Mar 2022', 'Feb 2022', 'Jan 2022', 'Dec 2021',
                    'Nov 2021', 'Oct 2021', 'Sep 2021', 'Aug 2021',
                    'July 2021', 'June 2021']
    )
)

# # chart line color red
# fig.update_traces(line_color='#FF0000')

fig.show()

In [3]:
%pip install dash

     |████████████████████████████████| 9.6 MB 3.7 MB/s eta 0:00:01
  Using cached Flask_Compress-1.11-py3-none-any.whl (7.9 kB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl (3.8 kB)
  Using cached dash_html_components-2.0.0-py3-none-any.whl (4.1 kB)
  Using cached dash_table-5.0.0-py3-none-any.whl (3.9 kB)
  Using cached Brotli-1.0.9-cp38-cp38-macosx_10_9_x86_64.whl (421 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install dash_bootstrap_components

  Using cached dash_bootstrap_components-1.0.3-py3-none-any.whl (209 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
from distutils.command.build import build
from dash import Dash, html, dcc, Output, Input, callback
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
# for sentiment analysis
import numpy as np
import re
from nltk.corpus import stopwords
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [22]:
def sentiment_analysis(chosen_defi_coin):
    stop = stopwords.words('english')
    def clean_content(contentInput):
        if type(contentInput) == np.float:
            return ""
        contentInput = contentInput.lower()
        contentInput = re.sub("'", "", contentInput) # to avoid removing contractions in english
        contentInput = re.sub("@[A-Za-z0-9_]+","", contentInput)
        contentInput = re.sub("#[A-Za-z0-9_]+","", contentInput)
        contentInput = re.sub(r'http\S+', '', contentInput)
        contentInput = re.sub('[()!?]', ' ', contentInput)
        contentInput = re.sub('\[.*?\]',' ', contentInput)
        contentInput = re.sub("[^a-z0-9]"," ", contentInput)
        contentInput = contentInput.strip()
        contentInput = contentInput.split()
        contentInput = [w for w in contentInput if not w in stop]
        contentInput = " ".join(word for word in contentInput)
        return contentInput
    string = chosen_defi_coin.lower() + ".csv"
    df = pd.read_csv(string)

    df['date'] = df['created'].apply(lambda x: x.split(' ')[0])
    df.sort_values(['date'], inplace=True)

    df['year_month'] = df['date'].apply(lambda x: x[0:x.rfind('-')])

    df = df.fillna(method='ffill')
    # monthList = pd.date_range('2021-01-01','2022-02-01', 
    #             freq='MS').strftime("%m/%Y").tolist()
    monthList = list(df['year_month'].unique())

    timeSplitData={}
    timeSplitDataSentiment = {}

    for month in monthList: 
        timeSplitData[month] = []
        timeSplitDataSentiment[month] = []  

    for row in df.iterrows():
        month = row[1]['year_month']
        content = row[1]['body']
        cleaned = clean_content(content)
        timeSplitData[month].append(cleaned)

    sid = SentimentIntensityAnalyzer()

    for month in monthList: # for month in timeSplitData:
        for content in timeSplitData[month]:
            output = sid.polarity_scores(content)
            timeSplitDataSentiment[month].append(output)
    
    timeSplitDataNetScore = {}
    for month in monthList:
        timeSplitDataNetScore[month] = 0 
        for output in timeSplitDataSentiment[month]:
            compound = output['compound']
            compound = compound / len(timeSplitDataSentiment[month])
            timeSplitDataNetScore[month] +=compound

    return timeSplitDataNetScore

    df = pd.DataFrame({'date': timeSplitDataNetScore.keys(), 'sentiment': timeSplitDataNetScore.values()})
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['date'], y=df['sentiment']))
    return fig

In [14]:
sentiment_analysis("maker")

<ipython-input-13-b33d724b7c22>:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if type(contentInput) == np.float:


In [20]:
from plotly.subplots import make_subplots

In [52]:
price_df = pd.read_csv("curve_price_data.csv")
# timeSplitDataNetScore = sentiment_analysis("curve")
sentiment_df = pd.DataFrame({'date': timeSplitDataNetScore.keys(), 'sentiment': timeSplitDataNetScore.values()})

def mdy_to_ymd(d):
    return datetime.strptime(d, '%b %d, %Y').strftime('%Y-%m-%d')

price_df['format_date'] = price_df['Date'].apply(lambda x:  mdy_to_ymd(x))

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=price_df['format_date'], y=price_df['Price'], name='Price'), secondary_y= False)
fig.update_xaxes(autorange="reversed") 
# fig.update_xaxes(dtick="M1")

fig.add_trace(go.Scatter(x=sentiment_df['date'], y=sentiment_df['sentiment'], name='Sentiment'), secondary_y= True)

fig.update_layout(
    title_text="Curve Data Price against Sentiment",
    title_x=0.5
)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Price", secondary_y=False)
fig.update_yaxes(title_text="Sentiment", secondary_y=True)

In [46]:
price_df['format_date'].unique()

array(['2021-12', '2021-11', '2021-10', '2021-09', '2021-08', '2021-07',
       '2021-06', '2021-05', '2021-04', '2021-03', '2021-02'],
      dtype=object)

In [50]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=price_df['Date'], y=price_df['Price'], name='Price'))
fig.update_xaxes(autorange="reversed") 

In [26]:
date = {'2021-01': 'Jan 01, 2021',
'2021-02': 'Feb 01, 2021',
'2021-03': 'Mar 01, 2021',
'2021-04': 'Apr 01, 2021',
'2021-05': 'May 01, 2021',
'2021-06': 'Jun 01, 2021',
'2021-07': 'Jul 01, 2021',
'2021-08': 'Aug 01, 2021',
'2021-09': 'Sep 01, 2021',
'2021-10': 'Oct 01, 2021',
'2021-11': 'Nov 01, 2021',
'2021-12': 'Dec 01, 2021',}

In [30]:
import plotly.express as px
df = px.data.stocks()
fig = px.line(df, x="date", y=df.columns,
            #   hover_data={"date": "|%B %d, %Y"},
              title='custom tick labels')
fig.update_xaxes(
    dtick="M1")
    # tickformat="%b\n%Y")
fig.show()

In [36]:
from datetime import datetime


def mdy_to_ymd(d):
    return datetime.strptime(d, '%b %d, %Y').strftime('%Y-%m-%d')

price_df['format_date'] = price_df['Date'].apply(lambda x:  mdy_to_ymd(x)[0: mdy_to_ymd(x).rfind('-')])

# mdy_to_ymd('Jan 01, 2021')
price_df.head(5)

,Date,Price,Open,High,Low,Vol.,Change %,format_date
0,"Dec 31, 2021",5.3268,5.1097,5.7109,5.0087,4.54M,4.17%,2021-12
1,"Dec 30, 2021",5.1137,4.8500,5.2745,4.6619,3.09M,5.78%,2021-12
2,"Dec 29, 2021",4.8342,4.7181,5.1932,4.7102,4.26M,2.36%,2021-12
3,"Dec 28, 2021",4.7229,5.2765,5.2873,4.6516,5.30M,-10.51%,2021-12
4,"Dec 27, 2021",5.2773,5.7339,5.7992,5.2729,2.47M,-7.90%,2021-12


In [35]:
sentiment_df.head(5)

,date,sentiment
0,2021-01,0.753102
1,2021-02,0.746449
2,2021-03,0.834420
3,2021-04,0.761834
4,2021-05,0.764964


In [54]:
%pip install dash_daq

  Using cached dash_daq-0.5.0.tar.gz (642 kB)
  Created wheel for dash-daq: filename=dash_daq-0.5.0-py3-none-any.whl size=669701 sha256=d37a12d3bb223e05ace80bf8ed7b3b591c245c4a0457d15c6375e3aab4f979e9
  Stored in directory: /Users/lamchingrou/Library/Caches/pip/wheels/00/35/e5/57e90f10c529601c6627400513cb65dd5adb09752411f5a050
Successfully built dash-daq
Note: you may need to restart the kernel to use updated packages.


In [57]:
import json
with open('sentimentalOutput/discord.json') as json_file:
    data = json.load(json_file)

In [59]:
type(data)

dict

In [1]:
import plotly.graph_objects as go

import networkx as nx

G = nx.random_geometric_graph(200, 0.125)

In [2]:
G